# HTR Kraken Cremma

Installation : 
- `pip install kraken`
- https://github.com/mittagessen/kraken
- https://kraken.re/main/index.html

MarketPlace on shell:
- kraken list
- kraken show <name>
- kraken get <name>

Library not fully matain :
- https://github.com/mittagessen/kraken/issues/573

In [1]:
import kraken

import cv2
from PIL import Image
import json
import matplotlib.pyplot as plt
from difflib import SequenceMatcher
import time

In [8]:
def Timer(name:str, start_time:float):
    actual_time = time.time()
    print(f"{name} {actual_time - start_time:.3f} sec")
    return actual_time

# Config

In [ ]:
# Install Python 3.10 on WSL on folder /kraken_env/
#   - no influence on python version out of WSL
#   - on WSL python3.10, pip and pip3.10 depend from python3.10
#   - on WSL python depend from python3.8
# If you have a build python 3.10 on Windows, this build will work independently from the first one
# But from now, when you will work on WSL, you will use Python 3.10 instead on 3.8

# Clic on this file from your Folder Explorer or run it from CMD Terminal
# Take a couple of minute to run 
# Must print "No broken requirements found." at the end

# Request WSL (sudo wsl --install)
wsl

# Creat folder
mkdir kraken_env
cd kraken_env/

# Update apt (just in case)
sudo apt update
sudo apt upgrade
sudo apt install build-essential zlib1g-dev libncurses5-dev libgdbm-dev libnss3-dev libssl-dev libreadline-dev libffi-dev wget

# Download & unzip folder Python3.10
wget https://www.python.org/ftp/python/3.10.0/Python-3.10.0.tgz
tar -xf Python-3.10.0.tgz
rm Python-3.10.0.tgz

# Configure & install the build
cd Python-3.10.0
./configure --enable-optimizations
make -j$(nproc)
sudo make altinstall

# Check the build is up
python3.10 --version

# Install kraken
pip3.10 install --upgrade pip
pip3.10 install kraken

# Check kraken has all the require lib
pip3.10 check kraken

# HTR

In [9]:
from kraken import binarization, pageseg

img_name = r'img_1.jpg'
#img = cv2.imread(r'image/' + img_name)
im = Image.open(r'image/' + img_name)
start_time = time.time()

bw_im = binarization.nlbin(im)
binarization_time = Timer("binarization.nlbin", start_time)

seg = pageseg.segment(bw_im)
pageseg_time = Timer("pageseg.segment", binarization_time)

binarization.nlbin 5.176 sec
pageseg.segment 48.616 sec


: 

In [ ]:
# https://colab.research.google.com/drive/1vjk-In0g9OKSRQJzqIG_F_Yq-xA8JPMp#scrollTo=sPiSz9isrVv6

from kraken import binarization, pagesegimport kraken
from kraken.lib import models

model = models.load_any("10.5281/zenodo.10592716")

In [ ]:
# script v1
from PIL import Image
from kraken.lib import vgsl
from kraken import blla
import serialization

MODEL_PATH = "./CREMMA-Medieval-LAT.mlmodel"
IMAGE_PATH = "/home/HTR/data/img_1.jpg"
OUTPUT_PATH = "/home/HTR/output.xml"

def process_image(image_filename, output_xml):
    try:
        im = Image.open(image_filename)
        seg_model = vgsl.TorchVGSLModel.load_model(MODEL_PATH)
        baseline_seg = blla.segment(im, model=seg_model)
        with open(output_xml, 'w') as fp:
            fp.write(serialization.serialize_segmentation(
                baseline_seg,
                image_name=image_filename,
                image_size=im.size,
                template='alto'
            ))
            print(f"Écriture de {output_xml}")
    except Exception as e:
        print(f"Erreur lors du traitement de {image_filename}. Erreur :{e}")

process_image(IMAGE_PATH, OUTPUT_PATH)

In [ ]:
# script v2
from PIL import Image
from kraken.binarization import nlbin
from kraken import ketos
import kraken.ketos.lib.pretrain.model

MODEL_PATH = "./CREMMA-Medieval-LAT.mlmodel"
IMAGE_PATH = "/home/HTR/data/img_1.jpg"
OUTPUT_PATH = "/home/HTR/output.xml"

im = Image.open(IMAGE_PATH)
binarized_image = nlbin(im)

model = ketos.pretrain()

binarized_image.save("binarized.jpg")
print("Done")